# Export Tenant Settings to Table
Currently there is no role which can only read the Fabric Tenant settings.*  
Nevertheless for the heavy users of the platform, e.g. Capacity Admins it would be interesting to know those settings.

Therefor this notebook proposes a workaround:
A user with the Fabric Admin role can schedule this notebook and can than give access to the resulting table which contains the current TenantSettings.

This can also be used to keep a change history of the settings in case this would be required by setting `keep_history` to True.
Another purpose of this could be to monitor for specific settings which might be changed without the actual intention, like [External Data Sharing](https://learn.microsoft.com/en-us/fabric/admin/service-admin-portal-export-sharing#external-data-sharing-preview)

*except for Global Reader in the overall tenant which would be oversharing: 
https://learn.microsoft.com/en-us/microsoft-365/admin/add-users/about-admin-roles?view=o365-worldwide#commonly-used-microsoft-365-admin-center-roles

In [ ]:
keep_history = True

In [ ]:
import requests
import pandas as pd
import datetime

token = mssparkutils.credentials.getToken('pbi')
headers = {
    'Authorization': f'Bearer {token}',
    'Content-type': 'application/json'
}

url = f"https://api.fabric.microsoft.com/v1/admin/tenantsettings"
print(url)
response = requests.get(url, headers=headers)

In [ ]:
df = pd.DataFrame(response.json()["tenantSettings"])
df["Export_Date"] = datetime.datetime.now()
display(df)

In [ ]:
sdf = spark.createDataFrame(df)

write_mode="overwrite"
if keep_history:
    write_mode = "append"

sdf.write.saveAsTable(name="Tenant_Settings", mode=write_mode)